# Part 1: Data Preprocessing

## 1. Data cleansing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

First need to import the data:

In [ ]:
path = "chimera_data_not_cleaned.csv"
df = pd.read_csv(path, sep = ",")
df.head()

Plot a histogram of the `age` column of the dataset. Do the values look reasonable to you? Do the same for `salary`.

In [ ]:
sns.histplot(df["age"],kde=False)
plt.show()

In [ ]:
sns.histplot(df["salary"],kde=False,bins=30)
plt.show()

Use the function `.unique()` to take a look at the values taken on by `education`, `years_since_promotion`, and `exit`.

In [ ]:
df["education"].unique()

In [ ]:
df["years_since_promotion"].unique()

In [ ]:
df['exit'].unique()

Use the function `.duplicated()` and conditioning to to detect if there are any duplicates.

In [ ]:
dups = df.duplicated() #checks each row of the dataset and returns TRUE or FALSE depending on whether it is a duplicate
print(dups.any()) #returns TRUE if there is any value in dups that is equal to TRUE
print(df[dups]) #returns the problematic row

Use the function `.drop_duplicates(inplace=True)` to obtain a new dataset with no duplicates.

In [ ]:
print(df.shape) #gives current size of dataset
df.drop_duplicates(inplace=True) # delete duplicate rows
print(df.shape)

In [ ]:
df.columns.duplicated()

Use the function `.isna()` and conditioning to detect if there are any empty cells.

In [ ]:
df.isna().any()

If we want to drop entire rows with NA values, we can simply use
`df.dropna(axis = 0,how="any")`

In [ ]:
df.dropna(axis = 0,how="any")

Use the function `.nunique()` to find the number of unique entries for all columns.

In [ ]:
df.nunique()

Drop any columns with only one value using `.drop(columns=['name1','name2'])`.

In [ ]:
df = df.drop(columns=['local'])
df.nunique()

## 2. Scaling/Normalization

In [ ]:
from sklearn import preprocessing

In [ ]:
X = np.array([[ 1., -1.,  2.],
                [ 2.,  0.,  0.],
                [ 0.,  1., -1.]])

An example of normalizing (putting the data in the [0,1] range).

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_minmax = min_max_scaler.fit_transform(X)
X_minmax

An example of scaling (making sure that the data has mean 0 and variance 1).

In [ ]:
X_scaled = preprocessing.scale(X)
X_scaled

## 3. Data imputation 

This can be done in Python using the following library. See the documentation at https://scikit-learn.org/stable/modules/impute.html

In [ ]:
from sklearn import impute

For example, we can replace missing values by the mean:

In [ ]:
X = np.array([[ 1., np.nan,  2.],
                [ 2.,  0.,  np.nan],
                [ 0.,  1., -1.]])

In [ ]:
imp = impute.SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
print(imp.transform(X))

In [ ]:
imp = impute.SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(df)
dfnew = pd.DataFrame(imp.transform(df))
dfnew.columns = df.columns
print(dfnew)

Do you see why this could be a problem? Try out the following:

In [ ]:
dfnew['exit'].unique()

For now, we will simply remove all rows with missing values

In [ ]:
print(df.shape)
df.dropna(axis=0,inplace=True)
print(df.shape)

## 4. Outliers

Using `seaborn`, plot a boxplot of `salary` as `exit` varies. Are there any outliers?

In [ ]:
sns.boxplot(data=df, x="exit",y="salary")
plt.show()

Using the function `stats.zscore(df[Column])` compute the z-score table for `salary`. Are there any outliers?

In [ ]:
from scipy import stats
Z = stats.zscore(df["salary"],nan_policy="omit") #compute z-score table
print(Z)

What data structure is obtained here? Find the index of the outlier in this case using `np.where` and logical conditions. Do the two indexes correspond?

In [ ]:
np.where((Z>3) | (Z<-3))

Let's now look at `boss_survey` as `exit` varies:

In [ ]:
sns.boxplot(data=df, x="exit",y="boss_survey")
plt.show()

Use the function `np.nanquantile(column,quantile)` to find the 5% quantile of `boss_survey` results within the employees exiting the firm. Then, take a look at all of the employees leaving the firm who have a `boss_survey` result at or below this 5% quantile.

In [ ]:
df_exiteers=df[df["exit"]==1]
lower_quantile_survey = np.nanquantile(df_exiteers["boss_survey"],0.05)
df_exiteers[df_exiteers["boss_survey"] <= lower_quantile_survey]

# Part 2: Feature Engineering

## 1. Numerical to Categorical
We start with ordinal then move onto one-hot encoding.

1. Ordinal encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

dftest = pd.DataFrame({'size':['small','medium','large','small','large','medium']})
dftest

In [ ]:
encoder = OrdinalEncoder(categories=[['small','medium','large']]) 
dfnew = encoder.fit_transform(dftest) # transform data
pd.DataFrame(data=dfnew, columns=dftest.columns)

2. One-hot encoding

In [ ]:
dftest = pd.DataFrame({'color':['green','red','red','blue','green','red']})
dftest

In [ ]:
pd.get_dummies(dftest,drop_first=True,columns=['color'])

## 2. Feature transforms

In [ ]:
dfcopy = df.copy()

Use `.apply(np.log)` to transform the column `salary` of `dfcopy` from itself to the log of itself.

In [ ]:
dfcopy['salary'] = dfcopy['salary'].apply(np.log)

Print both df and dfcopy. Check their `salary` column. Has your transform had the effect you wanted?

In [ ]:
df

In [ ]:
dfcopy

We can also make a graphical comparison:

In [ ]:
sns.histplot(df["salary"],kde=False,bins=30)
plt.show()

In [ ]:
sns.histplot(dfcopy["salary"],kde=False,bins=30)
plt.show()

Create a new feature in the dfcopy dataset consisting of the `boss_tenure_percentage`=`boss_tenure` / `tenure`. Then, print a histogram, using `seaborn`.

In [ ]:
dfcopy["boss_tenure_percentage"]=dfcopy["boss_tenure"]/dfcopy["tenure"]

In [ ]:
sns.histplot(dfcopy["boss_tenure_percentage"],kde=False,bins=15)
plt.show()

# Part 3: Exercises

## Exercise 1: Iceberg right ahead!

Next term, we will use the Titanic dataset, available at https://www.kaggle.com/c/titanic/data 
This is historic data containing the passengers present on the Titanic and some of their features (whether, e.g., they had family on board or not, their cabin numbers, etc.) and whether or not they survived the boat sinking.
Our goal is to clean up this dataset in view of using it later down the line. The dataset we will be cleaning up is `titanic_train.csv`.

In [ ]:
titanic_train=pd.read_csv("titanic_train.csv")

1. Observe the header of the dataset. What do SibSp and Parch represent?

In [ ]:
titanic_train.head()

2. Are there any duplicates in the dataset? Why are we doing this before dropping any columns?

In [ ]:
titanic_train.duplicated().any()

3. Drop a couple of columns from the datasets: PasssengerID, Name, and Ticket Number. We drop Ticket Number and PassengerId as they don't have much informative value. The Name could have some information in it (e.g., nobility, married or not, etc.) but that would require natural language processing, which we will not use on the dataset.

In [ ]:
titanic_train = titanic_train.drop(columns=["PassengerId","Name","Ticket"])

In [ ]:
titanic_train.head()

4. Let's check for inconsistencies in the numerical data using `.hist()`. Does anything seem abnormal to you?

In [ ]:
titanic_train.hist()
plt.show()

There may be some outliers but it seems like the values obtained are coherent.

5. What values do the categorical variables (this includes the cabin number) take on? Is there anything irregular there? Make sure you understand their output.

In [ ]:
titanic_train["Embarked"].unique()

This corresponds to the port at which the passenger embarked (e.g. S is for Southampton). There seems to be one or more missing values here.

In [ ]:
titanic_train["Cabin"].unique()

This corresponds to the cabin booked by the passenger. Some passengers seem to have booked many cabins at the same time. We check that this is indeed the case by taking a look at the fare paid for e.g. `B57 B59 B63 B66`.

In [ ]:
titanic_train[titanic_train["Cabin"]=="B57 B59 B63 B66"]

The fare is considerably more expensive so it tends to show that indeed, it was possible to book many cabins at once. There also seems to be one or more missing values here.

In [ ]:
titanic_train["Sex"].unique()

There seems to be no issue with this column.

6. We now deal with the missing values. Which features are missing information?

In [ ]:
titanic_train.isna().sum()

There seem to be an awful lot of cabins missing, as well as age, and embarcation port.

7. Which percentage of Age/Cabin/Embarked are missing? Use `.shape` to find this. In consequence, what should you with the Cabin column?

In [ ]:
titanic_train.shape

In [ ]:
177/891

In [ ]:
687/891

77% of the data in Cabin data is missing: this is a huge amount and probably not a very good predictor. We remove this column.

In [ ]:
titanic_train = titanic_train.drop(columns=["Cabin"])

8. For the Age column, we use a nearest neighbor approach. Use `KNNImputer` to fill in the missing values. Check that there are no more missing values in the Age column.

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)
titanic_train[["Age"]] = imputer.fit_transform(titanic_train[["Age"]])

In [ ]:
titanic_train.isna().sum()

9. For the Embarked column, use `countplot` in the `seaborn` package to obtain the number of people who embarked at `S`, `C`, and `Q`. Where did the overwhelming majority of passengers embark? Use `SimpleImputer` to simply replace all missing values with the most frequent one. Check that no more entries are missing.

In [ ]:
titanic_train.head()

In [ ]:
sns.countplot(x="Embarked", data=titanic_train)
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer

imp=SimpleImputer(missing_values=np.nan, strategy="most_frequent")
titanic_train[["Embarked"]]=imp.fit_transform(titanic_train[["Embarked"]])

In [ ]:
titanic_train.isna().sum()

10. The division into Parch and SibSp is quite random. We regroup this column into one column called `Family_Presence`. Create a new column in the dataframe called `Family_Presence` which contains 1 if either SibSp is equal to 1 or Parch is equal to 1. Then drop `Parch` and `SibSp`. Hint: Use `np.where(condition,1,0)` where `condition` is the logical condition needed to be satisfied.

In [ ]:
titanic_train["Family_Presence"]=np.where((titanic_train["SibSp"]>=1) | (titanic_train["Parch"]>=1), 1,0)

In [ ]:
titanic_train.head()

In [ ]:
titanic_train = titanic_train.drop(columns=["SibSp","Parch"])
titanic_train.head()

11. Finally, replace all categorical variables by numerical ones. We are ready to go!

In [ ]:
titanic_train = pd.get_dummies(titanic_train,drop_first=True)
titanic_train.head()

## Exercise 2:

Recall the notion of scaling and consider a feature for which we have many observations.

1. Show that if we take the feature vector, subtract its mean and divide by its standard deviation, then the new feature vector obtained is scaled, i.e., has mean 0 and standard deviation 1.

We subtract the mean $\mu$ of the feature vector from $x_i$ and divide by its standard deviation $\sigma$. In this way, each feature has now become $\frac{x_i-\mu}{\sigma}$. 
So, the average of this new feature vector is:
$$\frac{\frac{x_1-\mu}{\sigma}+\ldots+\frac{x_n-\mu}{\sigma}}{n}=\frac{x_1+\ldots+x_n-n \cdot \mu}{n \sigma}=\frac{1}{\sigma}\cdot \left( \frac{x_1+\ldots+x_n}{n} -\mu\right)=\frac{1}{\sigma}\cdot \left( \mu -\mu\right)=0.$$

Recall that the variance of numbers $y_1,\ldots,y_n$ is $$\frac{(y_1-\bar{y})^2+\ldots+(y_n-\bar{y})^2}{n}$$ where $\bar{y}$ is the average or mean of the numbers $y_1,\ldots,y_n$.
Here, we have just shown that the mean of the new feature vector is 0. Hence its variance is given by:
$$\frac{(\frac{x_1-\mu}{\sigma})^2+\ldots+(\frac{x_n-\mu}{\sigma})^2}{n}=\frac{1}{\sigma^2} \cdot \frac{(x_1-\mu)^2+\ldots+(x_n-\mu)^2}{n}=\frac{1}{\sigma^2} var(x_1,...,x_n)=\frac{1}{\sigma^2} \cdot \sigma^2=1.$$
As the standard deviation is the square root of the variance, it follows that the new feature vector has standard deviation one.

2. Check your answer on the first column of the np.array X below:

In [ ]:
X = np.array([[ 1., -1.,  2.],
              [ 2.,  0.,  0.],
              [ 0.,  1., -1.]])

In [ ]:
(X[:,0]-X[:,0].mean())/(X[:,0].std())
# we get the same thing as when we use the preprocessing library of sklearn

## Exercise 3: Motorcycle Helmets with Bluetooth

See the exercise description within Moodle!

1. We first create the two dataframes.

In [ ]:
dict_demand={'Price':[91,74.63,65.06,58.27,53.00],'Demand':[1000,2000,3000,4000,5000]}
df_demand=pd.DataFrame(data=dict_demand)
df_demand

In [ ]:
dict_supply={'Price':[9,23.21,31.53,37.42,42],'Supply':[1000,2000,3000,4000,5000]}
df_supply=pd.DataFrame(data=dict_supply)
df_supply

2. We now plot both dataframes on the same graph using seaborn. We want to plot a scatterplot so we use `sns.lineplot`. The curves do not intersect.

In [ ]:
sns.lineplot(x="Price", y="Demand", data=df_demand,color="red");
plot=sns.lineplot(x="Price", y="Supply", data=df_supply,color="blue");
plot.set_ylabel("Supply/Demand")
plt.show()

3. We add a new column to each dataframe by taking the log-transform of the supply/demand.

In [ ]:
df_supply["Log_supply"]=df_supply["Supply"].apply(log)
df_supply

In [ ]:
df_demand["Log_demand"]=df_demand["Demand"].apply(log)
df_demand

In [ ]:
sns.lineplot(x="Price", y="Log_demand", data=df_demand,color="red")
sns.lineplot(x="Price", y="Log_supply", data=df_supply,color="blue")
plt.show()

We get two lines.

4. We can estimate their slopes and intercepts quite easily: slopes=rise/run and intercept=y-axis - slope * x-axis.

In [ ]:
#slope of demand:
a=(df_demand["Log_demand"].loc[4]-df_demand["Log_demand"].loc[0])/(df_demand["Price"].loc[4]-df_demand["Price"].loc[0])
print(a)
#intercept of demand
b=df_demand["Log_demand"].loc[4]-a*df_demand["Price"].loc[4]
print(b)

In [ ]:
#slope of supply
c=(df_supply["Log_supply"].loc[4]-df_supply["Log_supply"].loc[0])/(df_supply["Price"].loc[4]-df_supply["Price"].loc[0])
print(c)
#intercept of demand
d=df_supply["Log_supply"].loc[4]-c*df_supply["Price"].loc[4]
print(d)

5. We can now solve for when they cross by setting $ap+b=cp+d$, i.e., $(a-c)p=d-b$ and so $p=(d-b)/(a-c)$.

In [ ]:
p=(d-b)/(a-c)
p

6. a. We construct two lists.

In [ ]:
L1=np.arange(0,31)/10
L1

In [ ]:
L2=np.exp(L1)
L2

In [ ]:
plt.plot(L1,L2)
plt.show()

6.b. The supply curve looks like an exponential function. The demand curve looks like a decreasing exponential function.

6.c. If $D=m\cdot e^{np}$ then taking the logarithm on both sides, 
$$\log(D)=\log(m\cdot e^{np})=\log(m)+\log(e^{np})=\log(m)+np$$ using basic rules of the logarithm and the fact that the exponential and logarithm are inverses of one another. Hence, $\log(D)$ is a linear function of the price $p$ and we have $a=n$ and $\log(m)=b$. A similar reasoning can be applied to $S$.